In [1]:
!pip install triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.6 MB/s eta 0:00:00


In [2]:
import torch
import triton
import triton.language as tl
from torch import tensor

@triton.jit
def compute_combinations_kernel(
    keys_ptr, areas_ptr, perimeters_ptr, bboxes_ptr, fourier_ptr, num_vertices_ptr, comb_keys_ptr, intersection_areas_ptr, union_areas_ptr,
    results1_ptr, results2_ptr, results3_ptr, results4_ptr, results5_1_ptr, results5_2_ptr, results6_ptr, results7_ptr,
    final_value_ptr, dictionary1_array_ptr,  # Pointer for expanded dictionary1 array
    n_combinations,  # Inputs and outputs
    BLOCK_SIZE: tl.constexpr, TENSOR_SIZE: tl.constexpr  # Block size and tensor size
):
    # Block ID and range of indices this block will handle
    pid = tl.program_id(0)
    idx = pid * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)
    mask = idx < n_combinations  # Ensure we stay within bounds

    # Load offsets for the combination key pairs
    offset1 = tl.load(comb_keys_ptr + idx * 2 + 0, mask=mask)
    offset2 = tl.load(comb_keys_ptr + idx * 2 + 1, mask=mask)

    #--------- CIRCULARITY SIMILARITY ------------#

    area_A = tl.load(areas_ptr + offset1, mask=mask)
    area_B = tl.load(areas_ptr + offset2, mask=mask)
    perimeter_A = tl.load(perimeters_ptr + offset1, mask=mask)
    perimeter_B = tl.load(perimeters_ptr + offset2, mask=mask)

    result1_1 = (4 * 3.141592653589793 * area_A) / (perimeter_A * perimeter_A)
    result1_2 = (4 * 3.141592653589793 * area_B) / (perimeter_B * perimeter_B)
    circularity_similarity= 1 / (1 + tl.abs(result1_1 - result1_2))


    #--------- PERIMETER SIMILARITY -----------#

    perimeter_similarity= 1 / (1 + tl.abs(perimeter_A - perimeter_B))

                  # Bounding Boxes
    list1_ptr = bboxes_ptr + offset1 * 4  # Each key has 4 items
    list2_ptr = bboxes_ptr + offset2 * 4

    # Polygon A bounding box
    list1_first_item = tl.load(list1_ptr + 0, mask=mask)
    list1_second_item = tl.load(list1_ptr + 1, mask=mask)
    list1_third_item = tl.load(list1_ptr + 2, mask=mask)
    list1_fourth_item = tl.load(list1_ptr + 3, mask=mask)

    # Polygon B bounding box
    list2_first_item = tl.load(list2_ptr + 0, mask=mask)
    list2_second_item = tl.load(list2_ptr + 1, mask=mask)
    list2_third_item = tl.load(list2_ptr + 2, mask=mask)
    list2_fourth_item = tl.load(list2_ptr + 3, mask=mask)


    #--------- ASPECT RATIO SIMILARITY -------------#

    aspect_ratio_A = (list1_third_item - list1_first_item) / (list1_fourth_item - list1_second_item)
    aspect_ratio_B = (list2_third_item - list2_first_item) / (list2_fourth_item - list2_second_item)
    aspect_ratio_sim= 1 / (1 + tl.abs(aspect_ratio_A - aspect_ratio_B))


    #---------- BOUNDING BOX DISTANCE --------------#

    center_Ax = (list1_first_item - list1_third_item) / 2
    center_Ay = (list1_second_item - list1_fourth_item) / 2

    center_Bx = (list2_first_item - list2_third_item) / 2
    center_By = (list2_second_item - list2_fourth_item) / 2

    diff_1 = center_Ax - center_Bx
    diff_2 = center_Ay - center_By
    bbox_distance_sim= 1 / (1 + tl.sqrt(diff_1 * diff_1 + diff_2 * diff_2))


    #----------- FOURIER SIMILARITY ----------------#

    tensor1_ptr = fourier_ptr + offset1 * TENSOR_SIZE
    tensor2_ptr = fourier_ptr + offset2 * TENSOR_SIZE

    sum_of_squares = tl.zeros([BLOCK_SIZE], dtype=tl.float32)
    for i in range(TENSOR_SIZE):
        tensor1_value = tl.load(tensor1_ptr + i, mask=mask)
        tensor2_value = tl.load(tensor2_ptr + i, mask=mask)
        diff = tensor1_value - tensor2_value
        diff_squared = diff * diff
        tl.store(dictionary1_array_ptr + idx * TENSOR_SIZE + i, diff_squared, mask=mask)
        if i>=1:
          val1 = tl.load(dictionary1_array_ptr + idx * TENSOR_SIZE + i, mask=mask)# Store squared difference
          val2 = tl.load(dictionary1_array_ptr + idx * TENSOR_SIZE + i-1, mask=mask)
          sum_of_squares += val1 + val2

    fourier_similarity= 1 / (1 + tl.sqrt(sum_of_squares))


    #-------------  JACARD SIMILARITY ----------------#

    intersection_area = tl.load(intersection_areas_ptr + idx, mask=mask)
    union_area = tl.load(union_areas_ptr + idx, mask=mask)
    jacard_similarity= intersection_area / union_area


    #-------------- AREA SIMILARITY -------------------#

    area_similarity= (2 * union_area) / (area_A + area_B)


    #--------------CURVATURE SIMILARITY --------------#

    value7_1 = tl.load(num_vertices_ptr + offset1, mask=mask)
    value7_2 = tl.load(num_vertices_ptr + offset2, mask=mask)
    curvature_similarity= tl.exp(-tl.abs(value7_1 - value7_2) / tl.maximum(value7_1, value7_2))

    # -------------------------------Combine Similarity Metrics----------------------------------------#

    final_value = 0.125 * (circularity_similarity+ perimeter_similarity+ jacard_similarity+ area_similarity+ aspect_ratio_sim+ bbox_distance_sim+ fourier_similarity+ curvature_similarity)

    # Store the results
    tl.store(results1_ptr + idx, circularity_similarity, mask=mask)
    tl.store(results2_ptr + idx, perimeter_similarity, mask=mask)
    tl.store(results3_ptr + idx, jacard_similarity, mask=mask)
    tl.store(results4_ptr + idx, area_similarity, mask=mask)
    tl.store(results5_1_ptr + idx, aspect_ratio_sim, mask=mask)
    tl.store(results5_2_ptr + idx, bbox_distance_sim, mask=mask)
    tl.store(results6_ptr + idx, fourier_similarity, mask=mask)
    tl.store(results7_ptr + idx, curvature_similarity, mask=mask)
    tl.store(final_value_ptr + idx, final_value, mask=mask)  # Store final_value


def compute_with_seventh_dir(keys, areas, perimeters, bboxes, fourier, num_vertices, key_lists, intersection_areas_dir, union_areas_dir):
    """
    Perform computations for all combinations of keys in each sublist, including results for directories.
    """
    all_combinations = []
    combination_counts = []
    total_combinations = 0
    intersection_areas_flat = []
    union_areas_flat = []

    # Generate all combinations for each sublist
    for i, sublist in enumerate(key_lists):
        combs = torch.combinations(torch.tensor(sublist, device='cuda'), r=2)
        all_combinations.append(combs)
        combination_counts.append(combs.shape[0])
        total_combinations += combs.shape[0]
        intersection_areas_flat.extend(intersection_areas_dir[i])
        union_areas_flat.extend(union_areas_dir[i])

    # Flatten all combinations
    all_combinations_flat = torch.cat(all_combinations).flatten()
    intersection_areas_flat = torch.tensor(intersection_areas_flat, device='cuda', dtype=torch.float32)
    union_areas_flat = torch.tensor(union_areas_flat, device='cuda', dtype=torch.float32)

    # Allocate result tensors
    results1 = torch.zeros(total_combinations, device='cuda', dtype=torch.float32)
    results2 = torch.zeros(total_combinations, device='cuda', dtype=torch.float32)
    results3 = torch.zeros(total_combinations, device='cuda', dtype=torch.float32)
    results4 = torch.zeros(total_combinations, device='cuda', dtype=torch.float32)
    results5_1 = torch.zeros(total_combinations, device='cuda', dtype=torch.float32)
    results5_2 = torch.zeros(total_combinations, device='cuda', dtype=torch.float32)
    results6 = torch.zeros(total_combinations, device='cuda', dtype=torch.float32)
    results7 = torch.zeros(total_combinations, device='cuda', dtype=torch.float32)
    final_value = torch.zeros(total_combinations, device='cuda', dtype=torch.float32)

    # Allocate memory for expanded dictionary1_array
    #dictionary1_array = torch.zeros(total_combinations * TENSOR_SIZE, device='cuda', dtype=torch.float32)

    # Launch the kernel
    BLOCK_SIZE = 128
    TENSOR_SIZE = 10  # Number of elements in each tensor in the sixth directory
    dictionary1_array = torch.zeros(total_combinations * TENSOR_SIZE, device='cuda', dtype=torch.float32)
    grid = (triton.cdiv(total_combinations, BLOCK_SIZE),)
    compute_combinations_kernel[grid](
        keys, areas, perimeters, bboxes, fourier, num_vertices, all_combinations_flat, intersection_areas_flat, union_areas_flat,
        results1, results2, results3, results4, results5_1, results5_2, results6, results7, final_value, dictionary1_array,
        n_combinations=total_combinations,
        BLOCK_SIZE=BLOCK_SIZE, TENSOR_SIZE=TENSOR_SIZE
    )

    # Split results back into sublists
    split_results1 = torch.split(results1, combination_counts)
    split_results2 = torch.split(results2, combination_counts)
    split_results3 = torch.split(results3, combination_counts)
    split_results4 = torch.split(results4, combination_counts)
    split_results5_1 = torch.split(results5_1, combination_counts)
    split_results5_2 = torch.split(results5_2, combination_counts)
    split_results6 = torch.split(results6, combination_counts)
    split_results7 = torch.split(results7, combination_counts)
    split_final_value = torch.split(final_value, combination_counts)

    return split_results1, split_results2, split_results3, split_results4, split_results5_1, split_results5_2, split_results6, split_results7, split_final_value


In [4]:
import numpy as np
import shapely
from shapely.geometry import Polygon
from shapely.affinity import translate
from shapely import intersection, union, get_exterior_ring, get_num_coordinates
import torch
from itertools import combinations



class ShapeSimilarity:
    def __init__(self):
        self.polygon_dict = {}
        self.area_dict = {}
        self.perimeter_dict = {}
        self.bbox_dict = {}
        self.cluster_similarities = []
        pass

    # Function to center polygons at the origin
    def center_polygons(self, polygon_dict):
        """
        Centers polygons in a dictionary by translating each polygon
        to have its centroid at the origin.

        Parameters:
            polygon_dict (dict): A dictionary where values are Shapely polygons.

        Returns:
            dict: A dictionary with the same keys but with centered polygons as values.
        """
        centered_polygon_dict = {}
        for key, polygon in polygon_dict.items():
            centroid = polygon.centroid
            centered_polygon = translate(polygon, xoff=-centroid.x, yoff=-centroid.y)
            centered_polygon_dict[key] = centered_polygon
        return centered_polygon_dict


    def sample_boundaries_vectorized(self, polygons, num_points=100):
        """
        Sample evenly spaced points along the exterior boundary of multiple polygons using Shapely's vectorized API.
        Args:
            polygons: Numpy array of shapely.geometry.Polygon objects.
            num_points: Number of points to sample per polygon.
        Returns:
            A (num_polygons, num_points, 2) numpy array of x, y coordinates.
        """
        # Extract exterior rings for all polygons
        exterior_rings = get_exterior_ring(polygons)

        # Generate normalized distances for evenly spaced sampling
        distances = np.linspace(0, 1, num_points)

        # Sample points from the exterior rings
        sampled_points = np.array([
            np.array([ring.interpolate(d * ring.length).coords[0] for d in distances])
            for ring in exterior_rings
        ])

        return sampled_points

    # Define PyTorch Fourier Descriptors Function
    def torch_fourier_descriptors(self, contours):
        """
        Compute Fourier Descriptors for a batch of contours using PyTorch.
        Args:
            contours: Tensor of shape (num_contours, num_points, 2).
        Returns:
            Tensor of shape (num_contours, num_descriptors).
        """
        complex_contours = torch.complex(contours[:, :, 0], contours[:, :, 1])
        fft_result = torch.fft.fft(complex_contours)
        return fft_result[:, :100]  # Take the first 10 descriptors


    # Step 2: Reconstruct the group_dict from flattened_pairs
    def create_group_dict_from_flattened(self, flattened_pairs, cluster_indices, group_sizes):
          """
          Create a dictionary mapping cluster indices to groups of pairs.

          Args:
              flattened_pairs (list): The 1D list of all pairs across groups.
              cluster_indices (list): The indices (keys) for each group.
              group_sizes (list): The number of pairs in each group.

          Returns:
              dict: A dictionary where each cluster index maps to a list of pairs for that group.
          """
          group_dict = {}
          start = 0  # Start index for slicing

          # Iterate through the group sizes and cluster indices
          for cluster_index, size in zip(cluster_indices, group_sizes):
              # Extract the pairs for this group
              group_dict[cluster_index] = flattened_pairs[start:start + size]
              start += size  # Move to the next group

          return group_dict



    def compute_fourier_descriptors(self, polygons, num_points=100):
        """
        Compute Fourier Descriptors for a set of polygons.

        Args:
            polygons: List or array of Shapely polygons.
            num_points: Number of points to sample along the boundaries.

        Returns:
            Fourier descriptors as a PyTorch tensor moved to the CPU.
        """
        # Sample evenly spaced points along the polygon boundaries
        sampled_contours = self.sample_boundaries_vectorized(polygons, num_points=num_points)

        # Convert contours to a PyTorch tensor
        contours_tensor = torch.tensor(sampled_contours, dtype=torch.float32)  # Shape: (num_polygons, num_points, 2)

        # Move to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        contours_tensor = contours_tensor.to(device)

        # Compute Fourier Descriptors
        fourier_descriptors = self.torch_fourier_descriptors(contours_tensor)

        # Move Fourier Descriptors to CPU
        return fourier_descriptors.cpu()



    def process_pairs_and_clusters_and_compute_areas(self, list_of_cluster_content_indices, cluster_indices_list):
        """
        Process pairs and clusters, generating unique pairs for each cluster,
        and compute intersection and union areas.

        Args:
            list_of_cluster_content_indices: List of cluster content indices (list of lists).
            cluster_indices_list: List of cluster indices corresponding to each cluster.

        Returns:
            flattened_pairs: Flattened list of unique pairs across all clusters.
            group_sizes: Sizes of groups corresponding to the clusters.
            intersection_areas: Numpy array of intersection areas for the flattened pairs.
            union_areas: Numpy array of union areas for the flattened pairs.
        """
        pairs = []
        group_sizes = []

        # For each group of keys
        for group_keys in list_of_cluster_content_indices:
            # Generate all unique pairs for this sublist
            sublist_pairs = list(combinations(range(len(group_keys)), 2))

            # Separate indices for polygons A and B
            idx_A, idx_B = np.array(sublist_pairs).T  # Separate indices for polygons A and B

            # Extract the corresponding polygons
            polygons_A = np.array([group_keys[i] for i in idx_A])

            polygons_B = np.array([group_keys[i] for i in idx_B])

            # Combine into pairs for bulk operations
            sublist_pairs = list(zip(polygons_A, polygons_B))

            # Append this sublist's pairs to the overall pairs structure
            pairs.append(sublist_pairs)
            group_sizes.append(len(sublist_pairs))

        # Create a dictionary where each key corresponds to a cluster index
        self.pairs_per_cluster = dict(zip(cluster_indices_list, pairs))
        #print(f"pairs_per_cluster: {self.pairs_per_cluster}")

        # Flatten the pairs into a 1D list
        flattened_pairs = [pair for sublist in pairs for pair in sublist]

        # Generate all unique pairs
        idx_A, idx_B = np.array(flattened_pairs).T  # Separate indices for polygons A and B

        # Extract the corresponding polygons
        polygons_A = np.array([self.polygon_dict[i] for i in idx_A])
        polygons_B = np.array([self.polygon_dict[i] for i in idx_B])

        # Combine into pairs for bulk operations
        flattened_pairs = list(zip(polygons_A, polygons_B))

        # Compute intersections and unions using bulk operations
        intersection_areas = shapely.area(intersection(*np.transpose(flattened_pairs)))
        union_areas = shapely.area(union(*np.transpose(flattened_pairs)))

        return flattened_pairs, group_sizes, intersection_areas, union_areas


    def transform_dict_and_sublists(self, input_dict, sublists):
        # Step 1: Sort the dictionary by its keys and get a list of (key, value) pairs
        sorted_items = sorted(input_dict.items())

        # Step 2: Create a new dictionary with keys as indices in the sorted order
        new_dict = {index: value for index, (_, value) in enumerate(sorted_items)}

        # Step 3: Create a mapping from old keys to new keys
        old_to_new_key_mapping = {old_key: index for index, (old_key, _) in enumerate(sorted_items)}

        # Step 4: Update the sublists using the mapping
        updated_sublists = [
            [old_to_new_key_mapping[key] for key in sublist]
            for sublist in sublists
        ]

        return new_dict, updated_sublists


    def sort_dictionary(self, input_dict):
        # Sort the dictionary by its keys
        sorted_items = sorted(input_dict.items())
        new_dict = {index: value for index, (_, value) in enumerate(sorted_items)}
        return new_dict


    def calculate_similarity_for_clusters(self, polygons, polygon_indices, list_of_cluster_content_indices, cluster_indices_list):
        """
        Main method to calculate similarity for multiple clusters.
        Args:
            polygons: List of shapely polygons.
            polygon_indices: List of indices representing the polygons.
            list_of_cluster_indices: List of cluster index lists (e.g., [[10, 30], [20, 40]]).
        Returns:
            List of average similarity scores, one for each cluster.
        """
        # Build polygon dictionary using provided polygon_indices. Each polygon correspond to its index
        self.polygon_dict = {index: polygon for index, polygon in zip(polygon_indices, polygons)}

        #Ignore dublicates
        self.polygon_dict = {key: (value[0] if isinstance(value, (list, tuple)) else value) for key, value in self.polygon_dict.items()}

        ########### Precompute all properties using vectorized operations #############

        #first center the polygons
        self.polygon_dict = self.center_polygons(self.polygon_dict)
        all_polygons = list(self.polygon_dict.values())

        new_dict, updated_sublists = self.transform_dict_and_sublists(self.polygon_dict, list_of_cluster_content_indices)

        #Precompute Areas of all Polygons
        print("Starting Coputing Areas...")
        all_areas = shapely.area(all_polygons)

        #Precompute Lengths of all Polygons
        print("Starting Computing Lengths")
        all_perimeters = shapely.length(all_polygons)

        #Precompute Bounds of all Polygons
        print("Starting Computing Bounds")
        all_bboxes = shapely.bounds(all_polygons)

        # Precompute number of exterior coordinates (vertices)
        print("Starting computing exterior coords")
        exterior_rings = get_exterior_ring(all_polygons)
        self.num_exterior_coords = shapely.get_num_coordinates(exterior_rings)

        # Precompute Fourier Descriptors
        print("Starting computing Fourier Discriptors:")
        self.fourier_descriptors = self.compute_fourier_descriptors(all_polygons)

        # Precompute Intersection Areas and Union Areas
        flattened_pairs, group_sizes,self.intersection_areas, self.union_areas = self.process_pairs_and_clusters_and_compute_areas(
            list_of_cluster_content_indices, cluster_indices_list
        )

        # Convert precomputed properties to dictionaries
        self.area_dict =  dict(zip(polygon_indices, all_areas))
        area_dict = self.sort_dictionary(self.area_dict)

        self.perimeter_dict = dict(zip(polygon_indices, all_perimeters))
        perimeter_dict = self.sort_dictionary(self.perimeter_dict)

        self.bbox_dict = dict(zip(polygon_indices, all_bboxes))
        bbox_dict = self.sort_dictionary(self.bbox_dict)

        self.num_exterior_coords_dict = dict(zip(polygon_indices, self.num_exterior_coords))
        num_exterior_coords_dict = self.sort_dictionary(self.num_exterior_coords_dict)

        self.fourier_descriptors_dict = dict(zip(polygon_indices, self.fourier_descriptors))
        fourier_descriptors_dict = self.sort_dictionary(self.fourier_descriptors_dict)

        self.intersection_areas_dict = self.create_group_dict_from_flattened(self.intersection_areas, list(range(len(cluster_indices_list))), group_sizes)
        intersection_areas_dict = self.sort_dictionary(self.intersection_areas_dict)

        self.union_areas_dict = self.create_group_dict_from_flattened(self.union_areas, list(range(len(cluster_indices_list))), group_sizes)
        union_areas_dict = self.sort_dictionary(self.union_areas_dict)


        # Convert dictionaries to GPU-compatible tensors
        keys = torch.tensor(list(area_dict.keys()), device='cuda', dtype=torch.int32)
        values1 = torch.tensor(list(area_dict.values()), device='cuda', dtype=torch.float32)
        values2 = torch.tensor(list(perimeter_dict.values()), device='cuda', dtype=torch.float32)
        values5 = torch.tensor([v for sublist in bbox_dict.values() for v in sublist], device='cuda', dtype=torch.float32)
        values6 = torch.cat([tensor for tensor in fourier_descriptors_dict.values()]).to('cuda').to(torch.float32)
        values7 = torch.tensor(list(num_exterior_coords_dict.values()), device='cuda', dtype=torch.float32)

        # Run the computation
        results1, results2, results3, results4, results5_1, results5_2, results6, results7, final_values= \
            compute_with_seventh_dir(
                keys, values1, values2, values5, values6, values7, updated_sublists, intersection_areas_dict, union_areas_dict
            )

        for i, sublist in enumerate(updated_sublists):

            # Compute the average of final values for this sublist
            average_final_value = final_values[i].mean().item()
            self.cluster_similarities.append(average_final_value)
        cluster_similarities_dict = dict(zip(cluster_indices_list, self.cluster_similarities))
        # Return the mapping of cluster indices to their respective average similarities
        return cluster_similarities_dict


# Create example polygons
polygons = [
    Polygon([(0, 0), (1, 0), (1, 1), (0, 1)]),   # Square
    Polygon([(0, 0), (2, 0), (2, 1), (0, 1)]),   # Rectangle
    Polygon([(0, 0), (1, 0), (0.5, 1)]),         # Triangle
    Polygon([(0, 0), (0.5, 0), (0.25, 0.5)]),    # Small Triangle
    Polygon([(0, 0), (1, 0), (0.5, 2)]),          # Tall Triangle
    Polygon([(0, 0), (1, 0), (1, 1), (0, 1)])
]

# Define indices and clusters
polygon_indices = [10, 20, 30, 40, 50, 10]                # Custom indices for the polygons
list_of_cluster_content_indices = [[10, 30, 40], [20, 50]]    # Two clusters to process
cluster_indices_list = [101, 200]  # Arbitrary indices for testing


# Instantiate and calculate
shape_sim = ShapeSimilarity()
similarity_scores = shape_sim.calculate_similarity_for_clusters(polygons, polygon_indices, list_of_cluster_content_indices, cluster_indices_list)
for cluster_indices, similarity in similarity_scores.items():
    print(f"Cluster {cluster_indices}: {similarity:.2f}%")


Starting Coputing Areas...
Starting Computing Lengths
Starting Computing Bounds
Starting computing exterior coords
Starting computing Fourier Discriptors:
Cluster 101: 0.78%
Cluster 200: 0.73%
